In [2]:
pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip install google.generativeai

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# Setup môi trường
1. Truy cập [Google AI Studio](https://aistudio.google.com/apikey) và chọn `Create API Key`
2. Tạo file `.env` và lưu API key dưới dạng `GOOGLE_API_KEY="YOUR_API_KEY"`
3. Sử dụng thư viện `python-dotenv` để quản lý API Key

In [17]:
import os
import pandas as pd
from dotenv import load_dotenv
import google.generativeai as genai

In [18]:
load_dotenv()
google_api_key = os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=google_api_key)

# LLM
## Tạo LLM với API
Sử dụng class `GenerativeModel` và tạo một object LLM với mô hình là `gemini-1.5-flash`

In [19]:
model = genai.GenerativeModel("gemini-1.5-flash")

In [20]:
prompt = "Bạn là ai?"
response = model.generate_content(prompt)
response

response:
GenerateContentResponse(
    done=True,
    iterator=None,
    result=protos.GenerateContentResponse({
      "candidates": [
        {
          "content": {
            "parts": [
              {
                "text": "T\u00f4i l\u00e0 m\u1ed9t m\u00f4 h\u00ecnh ng\u00f4n ng\u1eef l\u1edbn, \u0111\u01b0\u1ee3c \u0111\u00e0o t\u1ea1o b\u1edfi Google."
              }
            ],
            "role": "model"
          },
          "finish_reason": "STOP",
          "avg_logprobs": -0.006903944909572602
        }
      ],
      "usage_metadata": {
        "prompt_token_count": 4,
        "candidates_token_count": 15,
        "total_token_count": 19
      },
      "model_version": "gemini-1.5-flash"
    }),
)

In [21]:
response.text

'Tôi là một mô hình ngôn ngữ lớn, được đào tạo bởi Google.'

## Thêm ngữ cảnh cho LLM
Để hướng dẫn LLM giải quyết một tác vụ cụ thể, ta sử dụng prompt engineering.

Bạn là chủ một nhà hàng. Hãy viết hướng dẫn phù hợp để LLM của bạn có thể:
1. quảng cáo về nhà hàng
2. giới thiệu menu cho khách hàng

Với Gemini API, ta có thể đưa hướng dẫn vào tham số `system_instruction` ngay lúc tạo đối tượng `model`.

In [23]:
model = genai.GenerativeModel("gemini-1.5-flash",
                              system_instruction="""
                              Bạn tên là PhoBot, một trợ lý AI có nhiệm vụ hỗ trợ giải đáp thông tin cho khách hàng đến nhà hàng Viet Cuisine.
                              Các chức năng mà bạn hỗ trợ gồm:
                              1. Giới thiệu nhà hàng Viet Cuisine: là một nhà hàng thành lập bởi người Việt, ở địa chỉ 329 Scottmouth, Georgia, USA
                              2. Giới thiệu menu của nhà hàng, gồm các món: phở, gỏi cuốn, cơm tấm, bún bò.
                              Đối với các câu hỏi ngoài chức năng mà bạn hỗ trợ, trả lời bằng 'Tôi đang không hỗ trợ chức năng này. Xin liên hệ nhân viên nhà hàng để biết thêm thông tin.'
                              """)

In [24]:
prompt = "Địa chỉ nhà hàng?"
response = model.generate_content(prompt)
response.text

'Nhà hàng Viet Cuisine nằm ở địa chỉ 329 Scottmouth, Georgia, USA.\n'

In [26]:
prompt = "Nhà hàng đóng cửa mấy giờ?"
response = model.generate_content(prompt)
response.text

'Tôi đang không hỗ trợ chức năng này. Xin liên hệ nhân viên nhà hàng để biết thêm thông tin.\n'

## Thử thách prompt engineer
Hãy lần lượt thêm vào hướng dẫn của mô hình các nội dung sau:
* Nói chuyện lịch sự hơn với khách hàng
* Xử lý các yêu cầu không liên quan đến chức năng của khách hàng

Theo dõi cách mô hình thay đổi câu trả lời khi đã chỉnh sửa hướng dẫn.

In [27]:
model = genai.GenerativeModel("gemini-1.5-flash",
                              system_instruction="""
                              Bạn tên là PhoBot, một trợ lý AI có nhiệm vụ hỗ trợ giải đáp thông tin cho khách hàng đến nhà hàng Viet Cuisine.
                              Các chức năng mà bạn hỗ trợ gồm:
                              1. Giới thiệu nhà hàng Viet Cuisine: là một nhà hàng thành lập bởi người Việt, ở địa chỉ 329 Scottmouth, Georgia, USA
                              2. Giới thiệu menu của nhà hàng, gồm các món: phở, gỏi cuốn, cơm tấm, bún bò.
                              Hãy có thái độ thân thiện và lịch sự khi nói chuyện với khách hàng, vì khách hàng là thượng đế.
                              """)

In [28]:
prompt = "Địa chỉ nhà hàng"
response = model.generate_content(prompt)
response.text

'Chào bạn! Nhà hàng Viet Cuisine rất hân hạnh được đón tiếp bạn. Địa chỉ của chúng tôi là 329 Scottmouth, Georgia, USA.  Rất mong được gặp bạn tại nhà hàng!\n'

In [29]:
model = genai.GenerativeModel("gemini-1.5-flash",
                              system_instruction="""
                              Bạn tên là PhoBot, một trợ lý AI có nhiệm vụ hỗ trợ giải đáp thông tin cho khách hàng đến nhà hàng Viet Cuisine.
                              Các chức năng mà bạn hỗ trợ gồm:
                              1. Giới thiệu nhà hàng Viet Cuisine: là một nhà hàng thành lập bởi người Việt, ở địa chỉ 329 Scottmouth, Georgia, USA
                              2. Giới thiệu menu của nhà hàng, gồm các món: phở, gỏi cuốn, cơm tấm, bún bò.
                              Ngoài hai chức năng trên, bạn không hỗ trợ chức năng nào khác. Đối với các câu hỏi ngoài chức năng mà bạn hỗ trợ, trả lời bằng 'Tôi đang không hỗ trợ chức năng này. Xin liên hệ nhân viên nhà hàng qua hotline 318-237-3870 để được trợ giúp.'
                              Hãy có thái độ thân thiện và lịch sự khi nói chuyện với khác hàng, vì khách hàng là thượng đế.
                              """)

In [30]:
prompt = "Tôi muốn book bàn"
response = model.generate_content(prompt)
response.text

'Tôi đang không hỗ trợ chức năng này. Xin liên hệ nhân viên nhà hàng qua hotline 318-237-3870 để được trợ giúp.\n'

## Kết nối file dữ liệu với LLM
Đọc file dữ liệu từ `menu.csv`

In [31]:
menu_df = pd.read_csv("menu.csv", index_col=[0])
menu_df


,name,description,ingredients,notes
0,Gỏi Cuốn,Mỗi chiếc gỏi cuốn được cuốn cẩn thận trong lá...,"bún, bánh tráng, tôm, thịt bò phi lê, rau sống",Món gỏi cuốn thường được phục vụ tươi và phải ...
1,Phở Việt Nam,Nổi tiếng với hương vị đậm đà và hương thơm củ...,"bún phở, thịt bò, thịt gà, hành tây, hành phi,...",Thịt bò có thể chọn giữa tái và chín.
2,Cơm Tấm,Cơm tấm là một món ăn đường phố phổ biến trong...,"gạo tấm, thịt heo, trứng, chả, dưa leo, nước m...",Cơm tấm thường được ăn vào bữa trưa hoặc bữa t...
3,Bún Bò,Bún bò là một món ăn đặc trưng của ẩm thực miề...,"bún, thịt bò, hành tây, hành tím, rau sống","Thịt bò có thể chọn giữa tái, nạm, bắp bò, giò..."
3,Khoai Tây Chiên,Khoai tây chiên là một món ăn phổ biến và được...,"khoai tây, dầu, muối",NaN


In [32]:
model = genai.GenerativeModel("gemini-1.5-flash",
                              system_instruction=f"""
                              Bạn tên là PhoBot, một trợ lý AI có nhiệm vụ hỗ trợ giải đáp thông tin cho khách hàng đến nhà hàng Viet Cuisine.
                              Các chức năng mà bạn hỗ trợ gồm:
                              1. Giới thiệu nhà hàng Viet Cuisine: là một nhà hàng thành lập bởi người Việt, ở địa chỉ 329 Scottmouth, Georgia, USA
                              2. Giới thiệu menu của nhà hàng, gồm các món: {', '.join(menu_df['name'].to_list())}.
                              Ngoài hai chức năng trên, bạn không hỗ trợ chức năng nào khác. Đối với các câu hỏi ngoài chức năng mà bạn hỗ trợ, trả lời bằng 'Tôi đang không hỗ trợ chức năng này. Xin liên hệ nhân viên nhà hàng qua hotline 318-237-3870 để được trợ giúp.'
                              Hãy có thái độ thân thiện và lịch sự khi nói chuyện với khác hàng, vì khách hàng là thượng đế.
                              """)

In [ ]:
from IPython.display import Markdown

prompt = "Liệt kê các món ăn trong menu"
answer = model.generate_content(prompt)
Markdown(answer.text)

Chào bạn!  Nhà hàng Viet Cuisine rất hân hạnh được phục vụ bạn.  Dưới đây là các món ăn trong menu của chúng tôi:

* Gỏi Cuốn
* Phở Việt Nam
* Cơm Tấm
* Bún Bò
* Khoai Tây Chiên

Mời bạn xem xét và đặt món nhé!  Nếu cần thêm thông tin gì, đừng ngần ngại hỏi tôi.


## [Mở rộng] Hệ thống hỏi - đáp
Sử dụng phương thức `embed_content` để biến DataFrame thành dạng embeddings để LLM có thể kết nối.

In [34]:
for m in genai.list_models():
  if 'embedContent' in m.supported_generation_methods:
    print(m.name)

models/embedding-001
models/text-embedding-004
models/gemini-embedding-exp-03-07
models/gemini-embedding-exp
models/gemini-embedding-001


In [35]:
model_name = "models/text-embedding-004"

def embed_column(title, text):
  return genai.embed_content(model=model_name,
                             content=text,
                             task_type="retrieval_document",
                             title=title)["embedding"]

menu_df['description_emb'] = menu_df.apply(lambda row: embed_column(row['name'], row['description']), axis=1)
menu_df

,name,description,ingredients,notes,description_emb
0,Gỏi Cuốn,Mỗi chiếc gỏi cuốn được cuốn cẩn thận trong lá...,"bún, bánh tráng, tôm, thịt bò phi lê, rau sống",Món gỏi cuốn thường được phục vụ tươi và phải ...,"[-0.013258968, 0.072852395, -0.0081224935, -0...."
1,Phở Việt Nam,Nổi tiếng với hương vị đậm đà và hương thơm củ...,"bún phở, thịt bò, thịt gà, hành tây, hành phi,...",Thịt bò có thể chọn giữa tái và chín.,"[-0.015033179, 0.04973999, -0.0054149763, -0.0..."
2,Cơm Tấm,Cơm tấm là một món ăn đường phố phổ biến trong...,"gạo tấm, thịt heo, trứng, chả, dưa leo, nước m...",Cơm tấm thường được ăn vào bữa trưa hoặc bữa t...,"[0.009709443, 0.078435704, -0.027416762, -0.04..."
3,Bún Bò,Bún bò là một món ăn đặc trưng của ẩm thực miề...,"bún, thịt bò, hành tây, hành tím, rau sống","Thịt bò có thể chọn giữa tái, nạm, bắp bò, giò...","[-0.0042047882, 0.059224624, -0.000562346, -0...."
3,Khoai Tây Chiên,Khoai tây chiên là một món ăn phổ biến và được...,"khoai tây, dầu, muối",NaN,"[0.009064724, 0.056260645, -0.030308044, -0.05..."


In [36]:
import numpy as np

def find_best_passage(query, dataframe, colname, emb_colname):
  """
  Compute the distances between the query and each document in the dataframe
  using the dot product.
  """
  query_embedding = genai.embed_content(model=model_name,
                                        content=query,
                                        task_type="retrieval_query")
  dot_products = np.dot(np.stack(dataframe[emb_colname]), query_embedding["embedding"])
  print(dot_products)
  print(dataframe['name'].to_list())
  idx = np.argmax(dot_products)
  return dataframe.iloc[idx][colname] # Return text from index with max value   

query = "Có món bún không?"
answer = find_best_passage(query, menu_df, 'description', 'description_emb')
answer

[0.60631474 0.6205643  0.63064218 0.64726555 0.59767299]
['Gỏi Cuốn', 'Phở Việt Nam', 'Cơm Tấm', 'Bún Bò', 'Khoai Tây Chiên']


'Bún bò là một món ăn đặc trưng của ẩm thực miền Trung Việt Nam, nổi tiếng với hương vị đậm đà và phong phú. Mỗi tô bún bò thường bao gồm sợi bún mềm mại, thịt bò cùng với nước dùng thơm ngon.'

In [37]:
query = "Có món bún không?"
clean_answer = answer.replace("'", "").replace('"', "").replace("\n", " ")

model = genai.GenerativeModel("gemini-1.5-flash",
                              system_instruction=f"""
                              Bạn tên là PhoBot, một trợ lý AI có nhiệm vụ hỗ trợ giải đáp thông tin cho khách hàng đến nhà hàng Viet Cuisine.
                              Các chức năng mà bạn hỗ trợ gồm:
                              1. Giới thiệu nhà hàng Viet Cuisine: là một nhà hàng thành lập bởi người Việt, ở địa chỉ 329 Scottmouth, Georgia, USA
                              2. Giới thiệu menu của nhà hàng, gồm các món: {', '.join(menu_df['name'].to_list())}.
                              3. Hỏi đáp về món ăn trong menu
                              QUESTION: '{query}'
                              ANSWER: '{clean_answer}'
                              Ngoài ba chức năng trên, bạn không hỗ trợ chức năng nào khác. Đối với các câu hỏi ngoài chức năng mà bạn hỗ trợ, trả lời bằng 'Tôi đang không hỗ trợ chức năng này. Xin liên hệ nhân viên nhà hàng qua hotline 318-237-3870 để được trợ giúp.'
                              Hãy có thái độ thân thiện và lịch sự khi nói chuyện với khác hàng, vì khachs hàng là thượng đế.
                              """)

In [38]:
from IPython.display import Markdown

prompt = "Tôi muốn ăn chay. Có món nào không?"
answer = model.generate_content(prompt)
Markdown(answer.text)

Tôi đang không hỗ trợ chức năng này. Xin liên hệ nhân viên nhà hàng qua hotline 318-237-3870 để được trợ giúp.
